In [2]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

topic_model = BERTopic()
#topics, probs = topic_model.fit_transform(docs)

In [4]:
topics, probs = topic_model.fit_transform(docs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [7]:
topic_model.get_topic_info()


,Topic,Count,Name
0,-1,6256,-1_to_the_and_of
1,0,1841,0_game_team_games_he
2,1,582,1_key_clipper_encryption_chip
3,2,528,2_idjits_whatta_ites_cheek
4,3,418,3_drive_scsi_drives_ide
...,...,...,...
216,215,11,215_equation_rendering_pov_illumination
217,216,10,216_plutonium_nuclear_clancy_reactors
218,217,10,217_skin_greasy_acne_vaseline
219,218,10,218_clinton_bush_administration_choosing


In [9]:
topic_model.get_topic(0)

[('game', 0.010309179652224902),
 ('team', 0.008990615115314798),
 ('games', 0.007163778039114778),
 ('he', 0.00699129402973792),
 ('players', 0.006309604314460823),
 ('season', 0.006228576904751656),
 ('hockey', 0.0061055791904238805),
 ('play', 0.005761925564524764),
 ('25', 0.005624744309365813),
 ('year', 0.005585686456629487)]

In [26]:
test = topic_model.get_document_info(docs)
test.loc[3].to_csv('test.csv')